# NanoNet THEA

In [1]:
import os
import cv2
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 


from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, BatchNormalization, LayerNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D, ZeroPadding2D, Cropping2D
from tensorflow.keras.layers import UpSampling2D, SeparableConv2D, Input
from tensorflow.keras.layers import Add, Concatenate, Lambda, Reshape
from tensorflow.keras.metrics import Recall, Precision, MeanIoU
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from keras.utils import to_categorical
from se import squeeze_excite_block
from glob import glob




# Metics

In [2]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + tf.keras.losses.binary_crossentropy(y_true, y_pred)


# Reading Data 

In [3]:
H = 256
W = 256

def load_names(path, file_path):
    f = open(file_path, "r")
    data = f.read().split("\n")[:-1]
    images = [os.path.join(path,"images", name) + ".jpg" for name in data]
    masks = [os.path.join(path,"masks", name) + ".jpg" for name in data]
    return images, masks

def load_data(path):
    train_names_path = f"C:/thea/TheaNanoNet/DataSet/Kvasir-SEG/train.txt"
    valid_names_path = f"C:/thea/TheaNanoNet/DataSet/Kvasir-SEG/val.txt"

    train_x, train_y = load_names(path, train_names_path)
    valid_x, valid_y = load_names(path, valid_names_path)

    return (train_x, train_y), (valid_x, valid_y)

def read_image(path):
    path = path.decode()
    print("Reading image from path:", path)
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = tf.io.read_file(path)
    x = tf.image.decode_jpeg(x, channels=1)
    x = tf.image.resize(x, [H, W])
    x = tf.cast(x, tf.float32) / 255.0
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y


def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# Residual Block

In [4]:
def residual_block(x, num_filters, use_se_block=True):
    shortcut = x

    # first convolution layer
    x = Conv2D(filters=num_filters//4, kernel_size=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # second convolution layer
    x = Conv2D(filters=num_filters//4, kernel_size=3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # third convolution layer
    x = Conv2D(filters=num_filters, kernel_size=1, padding="same")(x)
    x = BatchNormalization()(x)

    # use 1x1 conv to match dimensions of x and shortcut if necessary
    if shortcut.shape[-1] != num_filters:
        shortcut = Conv2D(filters=num_filters, kernel_size=1, padding="same")(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # add shortcut and main path, and apply activation
    x = Add()([x, shortcut])
    x = Activation("relu")(x)

    # add squeeze and excitation block
    if use_se_block:
        x = squeeze_excite_block(x)

    return x


# Building Model

# NanoNet A

In [13]:
def NanoNet_A(img_height, img_width):
    f = [32, 64, 128]
    inputs = Input(shape=(img_height, img_width, 3), name="input_image")

    ## Encoder
    encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.50)
    encoder_output = encoder.get_layer(name="block_6_expand_relu").output
    skip_connections_name = ["input_image", "block_1_expand_relu", "block_3_expand_relu"]

    x = residual_block(encoder_output, 192)

    ## Decoder
    for i in range(1, len(skip_connections_name)+1, 1):
        x_skip = encoder.get_layer(skip_connections_name[-i]).output
        x_skip = Conv2D(f[-i], (1, 1), padding="same")(x_skip)
        x_skip = BatchNormalization()(x_skip)
        x_skip = Activation("relu")(x_skip)

        x = UpSampling2D((2, 2), interpolation='bilinear')(x)

        try:
            x = Concatenate()([x, x_skip])
        except Exception as e:
            x = Cropping2D(cropping=((1, 0), (0, 0)))(x)
            x = Concatenate()([x, x_skip])

        x = residual_block(x, f[-i])

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs=inputs, outputs=x)
    return model


# NanoNet B

In [6]:
def NanoNet_B(img_height, img_width):
    f = [32, 64, 96]
    inputs = Input(shape=(img_height, img_width, 3), name="input_image")

    ## Encoder
    encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.35)
    encoder_output = encoder.get_layer(name="block_6_expand_relu").output
    skip_connections_name = ["input_image", "block_1_expand_relu", "block_3_expand_relu"]

    x = residual_block(encoder_output, 128)

    ## Decoder
    for i in range(1, len(skip_connections_name)+1, 1):
        x_skip = encoder.get_layer(skip_connections_name[-i]).output
        x_skip = Conv2D(f[-i], (1, 1), padding="same")(x_skip)
        x_skip = BatchNormalization()(x_skip)
        x_skip = Activation("relu")(x_skip)

        x = UpSampling2D((2, 2), interpolation='bilinear')(x)

        try:
            x = Concatenate()([x, x_skip])
        except Exception as e:
            x = Cropping2D(cropping=((1, 0), (0, 0)))(x)
            x = Concatenate()([x, x_skip])

        x = residual_block(x, f[-i])

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# NanoNet C

In [7]:
def NanoNet_C(img_height, img_width):
    f = [16, 24, 32]
    inputs = Input(shape=(img_height, img_width, 3), name="input_image")

    ## Encoder
    encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.35)
    encoder_output = encoder.get_layer(name="block_6_expand_relu").output
    skip_connections_name = ["input_image", "block_1_expand_relu", "block_3_expand_relu"]

    x = residual_block(encoder_output, 48)

    ## Decoder
    for i in range(1, len(skip_connections_name)+1, 1):
        x_skip = encoder.get_layer(skip_connections_name[-i]).output
        x_skip = Conv2D(f[-i], (1, 1), padding="same")(x_skip)
        x_skip = BatchNormalization()(x_skip)
        x_skip = Activation("relu")(x_skip)

        x = UpSampling2D((2, 2), interpolation='bilinear')(x)

        try:
            x = Concatenate()([x, x_skip])
        except Exception as e:
            x = Cropping2D(cropping=((1, 0), (0, 0)))(x)
            x = Concatenate()([x, x_skip])

        x = residual_block(x, f[-i])

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

In [15]:
params = {}
params['img_height'] = 256
params['img_width'] = 256
params['img_channels'] = 3
params['mask_channels'] = 1

session = tf.compat.v1.Session()
graph = tf.compat.v1.get_default_graph()

with graph.as_default():
    with session.as_default():
        model = NanoNet_A(img_height=params["img_height"], img_width=params["img_width"])
        model.build(input_shape=(None, params["img_height"], params["img_width"], params["img_channels"]))
        model.summary()

        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

        # Optional: save printed results to file
        flops_log_path = 'files/tf_flops_log.txt'
        opts['output'] = 'file:outfile={}'.format(flops_log_path)

        # We use the Keras session graph in the call to the profiler.
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd='op', options=opts)

tf.compat.v1.reset_default_graph()
print(flops.total_float_ops)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 16  432         ['input_image[0][0]']            
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 16  64          ['Conv1[0][0]']                  
                                )                                                             

# Model Training

In [16]:
def create_dir(path):
    """ Create a directory. """
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"Error: creating directory with name {path}")

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_model_file(path):
    with CustomObjectScope({
            'iou':iou,
            'dice_coef':dice_coef,
            'dice_loss':dice_loss,
            'bce_dice_loss': bce_dice_loss
        }):
        model = tf.keras.models.load_model(path)
        return model


In [20]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Remove folders and files """
# os.system("rm files/files.csv")
# os.system("rm -r logs")


""" Hyperparameters """
img_height = 256
img_width = 256
img_channels = 3
mask_channels = 1

batch_size = 8
lr = 1e-4
epochs = 200
model_name = "TheaNanoNet_Model"
model_path = f"MODEL/{model_name}/model.h5"
csv_path = f"MODEL/{model_name}/model.csv"
log_path = f"logs/{model_name}/"

""" Creating folders """
create_dir(f"MODEL/{model_name}")

""" Dataset """
path = "C:/thea/TheaNanoNet/DataSet/Kvasir-SEG/" 
(train_x, train_y), (valid_x, valid_y) = load_data(path)

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

""" Model """
inputs = Input(shape=(img_height, img_width, img_channels), name="input_image")

## Encoder
encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.35)
encoder_output = encoder.get_layer(name="block_6_expand_relu").output
skip_connections_name = ["input_image", "block_1_expand_relu", "block_3_expand_relu"]

x = residual_block(encoder_output, 48)

## Decoder
f = [16, 24, 32]
for i in range(1, len(skip_connections_name)+1, 1):
    x_skip = encoder.get_layer(skip_connections_name[-i]).output
    x_skip = Conv2D(f[-i], (1, 1), padding="same")(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x_skip = Activation("relu")(x_skip)

    x = UpSampling2D((2, 2), interpolation='bilinear')(x)

    try:
        x = Concatenate()([x, x_skip])
    except Exception as e:
        x = Cropping2D(cropping=((1, 0), (0, 0)))(x)
        x = Concatenate()([x, x_skip])

    x = residual_block(x, f[-i])

## Output
x = Conv2D(mask_channels, (1, 1), padding="same")(x)
x = Activation("sigmoid")(x)

model = Model(inputs=inputs, outputs=x)
metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)
model.summary()

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(log_dir=log_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

train_steps = (len(train_x)//batch_size)
valid_steps = (len(valid_x)//batch_size)

if len(train_x) % batch_size != 0:
    train_steps += 1

if len(valid_x) % batch_size != 0:
    valid_steps += 1


model.fit(train_dataset, epochs=epochs, validation_data=valid_dataset, steps_per_epoch=train_steps, validation_steps=valid_steps, callbacks=callbacks, shuffle=False)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 16  432         ['input_image[0][0]']            
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 16  64          ['Conv1[0][0]']                  
                                )                                                           

UnknownError: Graph execution error:

Detected at node 'PyFunc' defined at (most recent call last):
    File "c:\Users\T470S\anaconda3\envs\thea\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\T470S\anaconda3\envs\thea\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "C:\Users\T470S\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\T470S\AppData\Roaming\Python\Python37\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "C:\Users\T470S\AppData\Roaming\Python\Python37\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\T470S\AppData\Roaming\Python\Python37\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\T470S\anaconda3\envs\thea\lib\asyncio\base_event

Reading image from path: C:/thea/TheaNanoNet/DataSet/Kvasir-SEG/images\cju1amqw6p8pw0993d9gc5crl.jpg
